### This method was very slow to run on my computer, and in fact would time out, so it was run with a reduced set of factors compared to the others. The rankings from the random forest runs were used as a guide. That was still very slow, so reduced number of rows. Finally, discovered the njobs parameter and could run 16 processes simultaneously. Each run here then took about three hours. So this wasn't investigated in too much depth.

## Some dependencies

In [ ]:
import pandas as pd
import numpy as np
import os

## Import and clean data

In [ ]:
exoplanets = pd.read_csv(os.path.join('Resources', 'cumulative.csv'))
pd.set_option('display.max_columns', None)
exoplanets

In [ ]:
#Remove rows (if any) where koi_pdisposition is not FALSE POSITIVE or CANDIDATE; 
#koi_disposition has additional categories
exoplanets.koi_pdisposition.unique()
#None found in current file

In [ ]:
#Make koi_pdisposition and koi_disposition numerical variables, see if they are the same (no, )
exoplanets_pdisp_cat = pd.get_dummies(exoplanets, prefix=['koi_pdisposition'], columns=['koi_pdisposition'])

In [ ]:
exoplanets_disp_cat = pd.get_dummies(exoplanets_pdisp_cat, prefix=['koi_disposition'], columns=['koi_disposition'])
exoplanets_disp_cat.drop('koi_pdisposition_FALSE POSITIVE', axis=1, inplace=True)

In [ ]:
exoplanets_basic = exoplanets_disp_cat.drop(["rowid", "kepoi_name", "kepler_name", "koi_score", 
                                             "koi_period_err1", "koi_period_err2", "koi_time0bk_err1", 
                                             "koi_time0bk_err2", "koi_impact_err1", "koi_impact_err2", 
                                             "koi_duration_err1", "koi_duration_err2", "koi_depth_err1", 
                                             "koi_depth_err2", "koi_prad_err1", "koi_prad_err2", "koi_teq_err1", 
                                             "koi_teq_err2", "koi_insol_err1", "koi_insol_err2", "koi_tce_plnt_num", 
                                             "koi_tce_delivname","koi_steff_err1", "koi_steff_err2", 
                                             "koi_slogg_err1", "koi_slogg_err2", "koi_srad_err1", "koi_srad_err2", 
                                             "koi_disposition_CANDIDATE", "koi_disposition_CONFIRMED", 
                                             "koi_disposition_FALSE POSITIVE"], axis=1)
exoplanets_basic.rename(columns={'koi_fpflag_nt': 'flag_not_transit_like', 
                   'koi_fpflag_ss': 'flag_stellar_eclipse', 
                   'koi_fpflag_co': 'flag_centroid_offset',
                   'koi_fpflag_ec': 'flag_ephemeris match',                  
                   'koi_period': 'orbital_period',                  
                   'koi_time0bk': 'time_first_trans_detected',
                   'koi_impact': 'star_planet_dist_at_conj',                   
                   'koi_duration': 'trans_duration',                   
                   'koi_depth': 'stellar_flux_loss_at_trans_min',
                   'koi_prad': 'planet_radius',
                   'koi_teq': 'approx_planet_temp',
                   'koi_insol': 'insolation_flux',
                   'koi_model_snr': 'trans_sig_to_noise',
                   'koi_steff': 'stellar_eff_temp',
                   'koi_slogg': 'stellar_surf_gravity',
                   'koi_srad': 'stellar_photosph_rad',
                   'ra': 'sky_location_right_asc',
                   'dec': 'sky_location_declination',                   
                   'koi_kepmag': 'stellar_magnitude'}, inplace=True)

exoplanets_basic.dropna(axis=0)
#None found

In [ ]:
np.any(np.isnan(exoplanets_basic))

In [ ]:
np.all(np.isfinite(exoplanets_basic))

In [ ]:
#https://stackoverflow.com/questions/31323499/sklearn-error-valueerror-input-contains-nan-infinity-or-a-value-too-large-for
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)
clean_dataset(exoplanets_basic)
exoplanets_basic

In [ ]:
np.any(np.isnan(exoplanets_basic))

In [ ]:
np.all(np.isfinite(exoplanets_basic))

In [ ]:
data_deflag = exoplanets_basic.drop(["kepid", 'flag_ephemeris match','flag_centroid_offset',
                                     'flag_stellar_eclipse','flag_not_transit_like' ], axis=1)

In [ ]:
data_grid = data_deflag.drop(['stellar_magnitude', 'sky_location_declination', 'stellar_surf_gravity', 
                              'sky_location_right_asc', 'stellar_photosph_rad', 'stellar_eff_temp', 
                              'time_first_trans_detected'], axis=1)

In [ ]:
data_grid_selection = data_grid.drop(data_grid.index[5001:9563])

## Grid Search

### First run: param_grid={'C': [1, 5, 10], 'gamma': [0.0001, 0.001, 0.01]}

In [ ]:
#target = exoplanets_basic["koi_pdisposition_CANDIDATE"].values.reshape(-1,1)
target = data_grid_selection["koi_pdisposition_CANDIDATE"]
target_names = ["False_positive", "Candidate"]

In [ ]:
data = data_grid_selection.drop(["koi_pdisposition_CANDIDATE"], axis=1)
feature_names = data.columns
data

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=33)

In [ ]:
# Support vector machine linear classifier
from sklearn.svm import SVC 
model = SVC(kernel='linear')

In [ ]:
# Create the GridSearch estimator along with a parameter object containing the values to adjust
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [1, 5, 10],
              'gamma': [0.0001, 0.001, 0.01]}
grid = GridSearchCV(model, param_grid, verbose=3, n_jobs=-1)

In [ ]:
# Fit the model using the grid search estimator. 
# This will take the SVC model and try each combination of parameters
grid.fit(X_train, y_train)

In [ ]:
# List the best parameters for this dataset
print(grid.best_params_)

In [ ]:
# Make predictions with the hypertuned model
predictions = grid.predict(X_test)


In [ ]:
print('Test Acc: %.3f' % grid.score(X_test, y_test))

In [ ]:
# Calculate classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions,
                            target_names=["False_positive", "Candidate"]))

First run summary:
    GridSearchCV(estimator=SVC(kernel='linear'), n_jobs=-1,
             param_grid={'C': [1, 5, 10], 'gamma': [0.0001, 0.001, 0.01]},
             verbose=3)
    -- best parameters were {'C': 1, 'gamma': 0.0001}, the lowest of each. Try even lower range. 
    
    Test Acc: 0.752
                        precision    recall  f1-score   support

False_positive       0.61      0.65      0.63       409
     Candidate       0.82      0.80      0.81       842

      accuracy                           0.75      1251
     macro avg       0.72      0.73      0.72      1251
  weighted avg       0.76      0.75      0.75      1251


## Second run: param_grid={'C': [0.1, 0.5, 1], 'gamma': [0.00001, 0.00005, 0.0001]}

In [ ]:
#target = exoplanets_basic["koi_pdisposition_CANDIDATE"].values.reshape(-1,1)
target = data_grid_selection["koi_pdisposition_CANDIDATE"]
target_names = ["False_positive", "Candidate"]
data = data_grid_selection.drop(["koi_pdisposition_CANDIDATE"], axis=1)
feature_names = data.columns

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=33)

# Support vector machine linear classifier
from sklearn.svm import SVC 
model = SVC(kernel='linear')

In [ ]:
# Create the GridSearch estimator along with a parameter object containing the values to adjust
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [0.1, 0.5, 1],
              'gamma': [0.00001, 0.00005, 0.0001]}
grid = GridSearchCV(model, param_grid, verbose=3, n_jobs=-1)


In [ ]:
# Fit the model using the grid search estimator. 
grid.fit(X_train, y_train)

In [ ]:
# List the best parameters for this dataset
print(grid.best_params_)

In [ ]:
# Make predictions with the hypertuned model
predictions = grid.predict(X_test)

In [ ]:
print('Test Acc: %.3f' % grid.score(X_test, y_test))

In [ ]:
# Calculate classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions,
                            target_names=["False_positive", "Candidate"]))

Second run summary

GridSearchCV(estimator=SVC(kernel='linear'), n_jobs=-1,
             param_grid={'C': [0.1, 0.5, 1], 'gamma': [1e-05, 5e-05, 0.0001]},
             verbose=3)

Best parameters: {'C': 0.1, 'gamma': 1e-05}

Test Acc: 0.771
    
                precision    recall  f1-score   support

False_positive       0.65      0.64      0.64       409
     Candidate       0.83      0.84      0.83       842

      accuracy                           0.77      1251
     macro avg       0.74      0.74      0.74      1251
  weighted avg       0.77      0.77      0.77      1251
  
  
 Best parameters were again the lowest, although improvement only marginal.

## Third run: param_grid={'C': [0.001, 0.05, 0.1], 'gamma': [0.000001, 0.000005, 0.00001]}

In [ ]:
#target = exoplanets_basic["koi_pdisposition_CANDIDATE"].values.reshape(-1,1)
target = data_grid_selection["koi_pdisposition_CANDIDATE"]
target_names = ["False_positive", "Candidate"]
data = data_grid_selection.drop(["koi_pdisposition_CANDIDATE"], axis=1)
feature_names = data.columns

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=33)

# Support vector machine linear classifier
from sklearn.svm import SVC 
model = SVC(kernel='linear')

In [ ]:
# Create the GridSearch estimator along with a parameter object containing the values to adjust
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [0.001, 0.05, 0.1],
              'gamma': [0.000001, 0.000005, 0.00001]}
grid = GridSearchCV(model, param_grid, verbose=3, n_jobs=-1)

In [ ]:
# Fit the model using the grid search estimator. 
grid.fit(X_train, y_train)

In [ ]:
# List the best parameters for this dataset
print(grid.best_params_)

In [ ]:
# Make predictions with the hypertuned model
predictions = grid.predict(X_test)

In [ ]:
print('Test Acc: %.3f' % grid.score(X_test, y_test))

In [ ]:
# Calculate classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions,
                            target_names=["False_positive", "Candidate"]))

Third run summary
GridSearchCV(estimator=SVC(kernel='linear'), n_jobs=-1,
             param_grid={'C': [0.001, 0.05, 0.1],
                         'gamma': [1e-06, 5e-06, 1e-05]},
             verbose=3)

Best parameters: {'C': 0.001, 'gamma': 1e-06}
    
Test Acc: 0.793
    
                precision    recall  f1-score   support

False_positive       0.86      0.44      0.58       409
     Candidate       0.78      0.97      0.86       842

      accuracy                           0.79      1251
     macro avg       0.82      0.70      0.72      1251
  weighted avg       0.81      0.79      0.77      1251


## Fourth run: param_grid={'C': [0.0001, 0.001, 0.01], 'gamma': [0.00000001, 0.0000001, 0.000001]}

In [ ]:
#target = exoplanets_basic["koi_pdisposition_CANDIDATE"].values.reshape(-1,1)
target = data_grid_selection["koi_pdisposition_CANDIDATE"]
target_names = ["False_positive", "Candidate"]
data = data_grid_selection.drop(["koi_pdisposition_CANDIDATE"], axis=1)
feature_names = data.columns

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=33)

# Support vector machine linear classifier
from sklearn.svm import SVC 
model = SVC(kernel='linear')

In [ ]:
# Create the GridSearch estimator along with a parameter object containing the values to adjust
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [0.0001, 0.001, 0.01],
              'gamma': [0.00000001, 0.0000001, 0.000001]}
grid = GridSearchCV(model, param_grid, verbose=3, n_jobs=-1)

In [ ]:
# Fit the model using the grid search estimator. 
grid.fit(X_train, y_train)

In [ ]:
# List the best parameters for this dataset
print(grid.best_params_)

In [ ]:
# Make predictions with the hypertuned model
predictions = grid.predict(X_test)

In [ ]:
print('Test Acc: %.3f' % grid.score(X_test, y_test))

In [ ]:
# Calculate classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions,
                            target_names=["False_positive", "Candidate"]))

Fourth run summary
GridSearchCV(estimator=SVC(kernel='linear'), n_jobs=-1,
             param_grid={'C': [0.0001, 0.001, 0.01],
                         'gamma': [1e-08, 1e-07, 1e-06]},

Best parameters: {'C': 0.01, 'gamma': 1e-08}
             
Test Acc: 0.797
             
                precision    recall  f1-score   support

False_positive       0.78      0.52      0.63       409
     Candidate       0.80      0.93      0.86       842

      accuracy                           0.80      1251
     macro avg       0.79      0.73      0.74      1251
  weighted avg       0.80      0.80      0.78      1251